In [11]:
import requests
import json

url = "https://google.serper.dev/scholar"

payload = json.dumps({
  "q": "Vaccination types and their efficacy in children under 5 years old",
})
headers = {
  'X-API-KEY': '340b3fd87989139a6c164e513092b2100a3d5987',
  'Content-Type': 'application/json'
}
print("Reached here")

response = requests.request("POST", url, headers=headers, data=payload)
  
try:
    # Use the POST request result that was already made
    if response.status_code == 200:
        data = response.json()
        results = []
        for paper in data.get("organic", []):  # Changed from "data" to "organic" based on actual response structure
            results.append({
                "source": "Google Scholar",
                "title": paper.get("title", ""),
                "snippet": paper.get("snippet", ""),  # Changed from "abstract" to "snippet" based on API response
                "link": paper.get("link", ""),  # Changed from "url" to "link" based on API response
                "year": paper.get("year", ""),
                "citations": paper.get("cited_by", {}).get("total", 0) if "cited_by" in paper else 0
            })
        print(f"Found {len(results)} papers")
    else:
        print(f"Error: API returned status code {response.status_code}")
        print(f"Response: {response.text}")
except Exception as e:
    print(f"Error processing response: {str(e)}")
# Print each paper's details
for i, paper in enumerate(results, 1):
    print(f"\nPaper {i}:")
    print(f"Title: {paper['title']}")
    print(f"Year: {paper['year']}")
    print(f"Citations: {paper['citations']}")
    print(f"Link: {paper['link']}")
    print(f"Abstract: {paper['snippet']}")
    print("-" * 80)

Reached here
Found 10 papers

Paper 1:
Title: Efficacy and effectiveness of influenza vaccination in healthy children. A review of current evidence
Year: 2023
Citations: 0
Link: https://www.sciencedirect.com/science/article/pii/S2529993X23000084
Abstract: … Although the WHO has recommended vaccination in children under 5 years of age since … the efficacy/effectiveness of influenza vaccination in healthy children <18 years of age through …
--------------------------------------------------------------------------------

Paper 2:
Title: Efficacy of pneumococcal vaccination in children younger than 24 months: a meta-analysis
Year: 2009
Citations: 0
Link: https://publications.aap.org/pediatrics/article-abstract/123/6/e1103/71593
Abstract: … Effectiveness of heptavalent pneumococcal conjugate vaccine in children younger than 5 years of age for prevention of pneumonia: updated analysis using World Health Organization …
-------------------------------------------------------------------------

In [4]:
from fastapi import FastAPI, HTTPException, Depends
from uuid import UUID
from datetime import datetime
import logging
from typing import List, Dict, Any
from pydantic import BaseModel

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create FastAPI app instance
app = FastAPI()

# Define response model
class AIInsightResponse(BaseModel):
    timestamp: str
    disclaimer: str
    insights: str
    data_gaps: Dict[str, Any]
    methodology_note: str

# Define session class
class SessionInfo:
    def __init__(self):
        self.data: Dict[str, Any] = {}

# Define the AI insights prompt template
AI_INSIGHTS_PROMPT = """
Based on the following research questions and identified data gaps, please provide comprehensive insights:

Questions:
{main_questions}

Data Gaps:
{data_gaps}

Web Results:
{web_results}

Please provide:
1. Analysis of available information
2. Preliminary answers to research questions
3. Identification of key findings
4. Areas requiring further investigation
"""

# Helper functions
async def get_active_session(session_id: UUID) -> SessionInfo:
    # This is a mock implementation
    return SessionInfo()

async def analyze_gaps(session_id: UUID, session: SessionInfo):
    # This is a mock implementation
    return {"identified_gaps": ["gap1", "gap2"]}

async def fetch_webpage(urls: List[str], query: str):
    # Use the existing results from previous cell
    return results if 'results' in globals() else []

def get_llm(temperature: float = 0.7):
    # Mock LLM class
    class MockLLM:
        async def invoke(self, prompt):
            class Response:
                content = "Mock AI-generated insights based on the provided information."
            return Response()
    return MockLLM()

print(AIInsightResponse)

<class '__main__.AIInsightResponse'>


In [5]:

# --- Step 5: AI Research Insights ---
@app.post("/research/analyze/ai-insights", 
          response_model=AIInsightResponse,
          tags=["Research Analysis"])
async def get_ai_research_insights(
    session_id: UUID,
    session: SessionInfo = Depends(get_active_session)
):
    """
    Provide AI-generated insights and preliminary answers for research questions.
    
    This endpoint:
    1. Uses identified data gaps to understand limitations
    2. Searches web sources for relevant information
    3. Generates preliminary answers and insights
    4. Provides citations and confidence levels
    5. Suggests areas for further research
    
    Returns a structured response with:
    - Timestamp of analysis
    - Important disclaimers
    - AI-generated insights
    - Data gap analysis
    - Methodology notes
    """
    try:
        logger.info(f"Generating AI research insights for session {session_id}")
        
        if "questions" not in session.data:
            raise HTTPException(status_code=400, detail="Generate questions first")
            
        if "data_gaps" not in session.data:
            # First analyze data gaps by calling the gaps endpoint
            questions = session.data["questions"]
            gaps_response = await analyze_gaps(session_id, session)
            session.data["data_gaps"] = gaps_response  # analyze_gaps already updates the session
            
        # Get the LLM instance with research-focused system prompt
        llm = get_llm(temperature=0.7)  # Slightly higher temperature for more comprehensive responses
        
        # Prepare context from questions and gaps
        main_questions = session.data["questions"].get("main_questions", [])
        data_gaps = session.data["data_gaps"]
        
            # Search the web for relevant information
        all_web_results = []
        for question in main_questions:
            # Make sure we're using the question text, not the dict
            question_text = question.get("question", "")
            if question_text:
                web_results = await fetch_webpage(
                    urls=[], # The tool will search automatically
                    query=question_text
                )
                if web_results:
                    all_web_results.extend(web_results)
        
        # Create comprehensive prompt for the LLM
        prompt = AI_INSIGHTS_PROMPT.format(
            main_questions=[q.get("question", "") for q in main_questions],
            data_gaps=data_gaps,
            web_results=all_web_results
        )
        
        response = await llm.invoke(prompt)
        
        # Structure the insights
        insights = {
            "timestamp": datetime.now().isoformat(),
            "disclaimer": """IMPORTANT: These insights are AI-generated based on available information and should be 
                        considered preliminary. All findings should be verified through rigorous research and peer-reviewed sources.""",
            "insights": response.content,
            "data_gaps": data_gaps,
            "methodology_note": """This analysis combines web-sourced information with AI interpretation. 
                               Statistics and claims should be independently verified."""
        }
        
        # Store in session for future reference
        session.data["ai_insights"] = insights
        
        logger.info(f"Successfully generated AI insights for session {session_id}")
        return insights
        
    except Exception as e:
        logger.error(f"Error generating AI insights: {str(e)}")
        raise HTTPException(
            status_code=500,
            detail=f"Error generating AI insights: {str(e)}"
        )
    #Print results from the API.
print("Successfully generated AI insights:")

Successfully generated AI insights:
